# Sensitivity analysis for non-parametric causal estimators
Sensitivity analysis helps us study how robust an estimated effect is when the assumption of no unobserved confounding is violated. That is, how much bias does our estimate have due to omitting  an (unobserved) confounder? Known as the 
*omitted variable bias (OVB)*, it gives us a measure of how the inclusion of an omitted common cause (confounder) would have changed the estimated effect. 

This notebook shows how to estimate the OVB for general, non-parametric causal estimators. For gaining intuition, we suggest going through an introductory notebook that describes how to estimate OVB for a a linear estimator: [Sensitivity analysis for linear estimators](https://github.com/py-why/dowhy/blob/master/docs/source/example_notebooks/sensitivity_analysis_testing.ipynb). To recap, in that notebook, we saw how the OVB depended on linear partial R^2 values and used this insight to compute the adjusted estimate values depending on the relative strength of the confounder with the outcome and treatment. We now generalize the technique using the non-parametric partial R^2 and Reisz representers.


This notebook is based on Chernozhukov et al., Long Story Short: Omitted Variable Bias in Causal Machine Learning.  https://arxiv.org/abs/2112.13398. 

## I. Sensitivity analysis for partially linear models
We first analyze the sensitivity of a causal estimate when the true data-generating process (DGP) is known to be partially linear. That is, the outcome can be additively decomposed into a linear function of the treatment and a non-linear function of the confounders. 
$$ Y = g(T, W, U) + \epsilon = \theta T + h(W, U) + \epsilon $$

However, we cannot estimate the above equation because the confounders $U$ are unobserved. Thus, in practice, a causal estimator uses the following "short" equation, 
$$ Y = g_s(T, W) + \epsilon = \theta_s T + h_s(W) + \epsilon_s $$

The goal of sensitivity analysis is to answer how far $\theta_s$ would be from the true $\theta$. Chernozhukov et al. show that given a special function called Reisz function $\alpha$, the omitted variable bias, $|\theta - \theta_s|$ is bounded by $\sqrt{E[g-g_s]^2E[\alpha-\alpha_s]^2}$. For partial linear models, $\alpha$ and the "short" $\alpha_s$  are defined as, 
$$ \alpha := \frac{T - E[T | W, Z] )}{(E(T - E[T | W, Z]) ^ 2)}$$
$$ \alpha_s := \frac{(T - E[T | W] )}{(E(T - E[T | W]) ^ 2)} $$

The bound can be expressed in terms of the *partial* R^2 of the unobserved confounder $U$ with the treatment and outcome. Recall that R^2 of $U$ wrt some target $Z$ is defined as the ratio of variance of the prediction $E[Z|U]$ with the variance of $Z$, $R^2_{Z\sim U}=\frac{Var(E[Z|U])}{Var(Y)}$. We can define the partial R^2 as an extension that measures the additional gain in explanatory power conditioned on some variables $W$. 
$$ \eta^2_{Z\sim U| W} = \frac{Var(E[Z|W, U]) - Var(E[Z|W])}{Var(Z) - Var(E[Z|W])} $$

The bound is given by, 
$$ (\theta - \theta_s)^2 = E[g-g_s]^2E[\alpha-\alpha_s]^2 = S^2 C_Y^2 C_T^2 $$ 
where, 
$$ S^2 = \frac{E[(Y-g_s)^2]}{E[\alpha_s^2]}; \ \ C_Y^2 = \eta^2_{Y \sim U | T, W}, \ \ C_T^2 = \frac{\eta^2_{T\sim U | W}}{1 - \eta^2_{T\sim U | W}}$$


$S^2$ can be estimated from data. The other two parameters need to be specified manually: they convey the strength of the unobserved confounder $U$ on treatment and outcome. Below we show how to create a contour plot by specifying a range of values for $\eta^2_{Y \sim U | T, W}$ and $\eta^2_{T\sim U | W}$. We also show how to benchmark and set these values as a fraction of the maximum partial R^2 due to any subset of the observed covariates. 

### Creating a dataset with unobserved confounding 

In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
# Required libraries
import re
import numpy as np
import dowhy
from dowhy import CausalModel
import dowhy.datasets
from dowhy.utils.util import create_polynomial_function

We create a dataset with linear relationship between treatment and outcome, following the partial linear data-generating process. $\beta$ is the true causal effect.

In [ ]:
np.random.seed(101) 
data = dowhy.datasets.partially_linear_dataset(beta = 10,
                                               num_common_causes = 7,
                                               num_unobserved_common_causes=1,
                                               strength_unobserved_confounding=10,
                                               num_samples = 500,
                                               num_treatments = 1,
                                               stddev_treatment_noise = 10,
                                               stddev_outcome_noise = 5
                                                )
display(data)

The true ATE for this data-generating process is,

In [ ]:
data["ate"]

To simulate unobserved confounding, we remove one of the common causes from the dataset. 


In [ ]:
# Observed data 
dropped_cols=["W0"]
user_data = data["df"].drop(dropped_cols, axis = 1)
# assumed graph
user_graph = data["gml_graph"]
for col in dropped_cols:
    user_graph = user_graph.replace('node[ id "{0}" label "{0}"]'.format(col), '')
    user_graph = re.sub('edge\[ source "{}" target "[vy][0]*"\]'.format(col), "", user_graph)
user_data

### Obtaining a causal estimate using Model, Identify, Estimate steps
Create a causal model with the "observed" data and causal graph.

In [ ]:
#graph_str = 'graph[directed 1node[ id "y" label "y"]node[ id "W0" label "W0"] node[ id "W1" label "W1"] node[ id "W2" label "W2"] node[ id "W3" label "W3"]  node[ id "W5" label "W5"] node[ id "W6" label "W6"]node[ id "v0" label "v0"]edge[source "v0" target "y"]edge[ source "W0" target "v0"] edge[ source "W1" target "v0"] edge[ source "W2" target "v0"] edge[ source "W3" target "v0"] edge[ source "W5" target "v0"] edge[ source "W6" target "v0"]edge[ source "W0" target "y"] edge[ source "W1" target "y"] edge[ source "W2" target "y"] edge[ source "W3" target "y"] edge[ source "W5" target "y"] edge[ source "W6" target "y"]]'
model = CausalModel(
            data=user_data,
            treatment=data["treatment_name"],
            outcome=data["outcome_name"],
            graph=user_graph,
            test_significance=None,
        )
model.view_model()
from IPython.display import Image, display
display(Image(filename="causal_model.png"))

In [ ]:
# Identify effect
identified_estimand = model.identify_effect(proceed_when_unidentifiable=True)
print(identified_estimand)

In [ ]:
# Estimate effect
import econml
from sklearn.ensemble import GradientBoostingRegressor
linear_dml_estimate = model.estimate_effect(identified_estimand, 
                                    method_name="backdoor.econml.dml.LinearDML",
                                    method_params={
                                        'init_params': {'model_y':GradientBoostingRegressor(),
                                                        'model_t': GradientBoostingRegressor(),
                                                        'linear_first_stages': False
                                                       },
                                        'fit_params': {'cache_values': True,}
                                     })

### Sensitivity Analysis using the Refute step
After estimation , we need to check how robust our estimate is against the possibility of unobserved confounders.  We perform sensitivity analysis for the LinearDML estimator assuming that its assumption on data-generating process holds: the true function for $Y$ is partial linear. 

Note that partial linear sensitivity analysis is automatically chosen if LinearDML estimator is used for estimation. For computational efficiency, you can set <b>cache_values</b> = <b>True</b> in `fit_params` to cache the results of first stage estimation. Parameters used:

* <b>method_name</b>: Refutation method name <br>
* <b>simulated_method_name</b>: "non-parametric-partial-R2" for non Parametric Sensitivity Analysis<br>
* **effect_strength_on_treatment**: $\eta^2_{T\sim U | W}$, Partial R2 of unobserved confounder with treatment conditioned on all observed confounders. 
* **effect_strength_on_outcome**: $\eta^2_{Y \sim U | T, W}$, Partial R2 of unobserved confounder with outcome conditioned on treatment and all observed confounders. 

In [ ]:
refute = model.refute_estimate(identified_estimand, linear_dml_estimate ,
                               method_name = "add_unobserved_common_cause",
                               simulation_method = "non-parametric-partial-R2",
                               effect_strength_on_treatment = np.arange(0, 0.8, 0.1),
                               effect_strength_on_outcome = np.arange(0, 0.8, 0.1)
                              )
print(refute)

**Intepretation of the plot.** In the above plot, the x-axis shows hypothetical partial R2 values of unobserved confounder(s) with the treatment. The y-axis shows hypothetical partial R2 of unobserved confounder(s) with the outcome. At <x=0,y=0>, the black diamond shows the original estimate (theta_s) without considering the unobserved confounders.

The contour levels represent *adjusted* lower confidence bound estimate of the effect, which would be obtained if the unobserved confounder(s) had been included in the estimation model. The red contour line is the critical threshold where the adjusted effect goes to zero. Thus,  confounders with such strength or stronger are sufficient to reverse the sign of the estimated effect and invalidate the estimate's conclusions.

In [ ]:
refute.RV

The robustness value measures the minimal equal strength of $\eta^2_{T\sim U | W}$ and $\eta^2_{Y \sim U | T, W}$ such the bound for the average treatment effect would include zero. <br>
A robustness value of 0.4 implies that confounders with $\eta^2_{T\sim U | W}$ and $\eta^2_{Y \sim U | T, W}$ values less than 0.4 would not be sufficient enough to bring down the estimates to zero.

**Benchmarking.** In general, however, providing a plausible range of partial R2 values is difficult. Instead,  we can infer the partial R2 of the unobserved confounder as a multiple of the partial R2 of any subset of observed confounders. So now we just need to specify the effect of unobserved confounding as a multiple/fraction of the observed confounding. This process is known as *benchmarking*.

The relevant arguments for the method are described below. 
- <b>benchmark_common_causes</b>: Name of the covariates used to bound the strengths of unobserved confounder<br>
- <b>effect_fraction_on_treatment</b>: Strength of association between unobserved confounder and treatment compared to benchmark covariate<br>
- <b>effect_fraction_on_outcome</b>: Strength of association between unobserved confounder and outcome compared to benchmark covariate<br>

In [ ]:
refute_bm = model.refute_estimate(identified_estimand, linear_dml_estimate ,
                               method_name = "add_unobserved_common_cause",
                               simulation_method = "non-parametric-partial-R2",
                               benchmark_common_causes = ["W1"],
                               effect_fraction_on_treatment = 0.2,
                               effect_fraction_on_outcome = 0.2
                              )

The red triangle shows the estimated partial-R^2 of a chosen benchmark observed covariate with the treatment and outcome. In the above call, we chose *W1* as the benchmark covariate. Under the unobserved confounder cannot be stronger in its effect on treatment and outcome than the observed benchmark covariate (*W1*), the above plot shows that the mean estimated effect will change after accounting for unobserved confounding.


**Plot types**. The default `plot_type` is to show the `lower_confidence_bound` under a significance level . Other possible values for the `plot_type` are:
* `upper_confidence_bound`: in case the unobserved confounder is expected to lower the estimate.
* `lower_ate_bound`: lower (point) estimate for unconfounded average treatment effect without considering the significance level
* `upper_ate_bound`: upper (point) estimate for unconfounded average treatment effect without considering the significance level
* `bias`: the bias of the obtained estimate compared to the true estimate

In [ ]:
refute_bm.plot(plot_type = "upper_confidence_bound")
refute_bm.plot(plot_type = "bias")

We can also access the refutation results as a data frame.

In [ ]:
refute_bm.results

## II. Sensitivity Analysis for general non-parametric models
We now perform sensitivity analysis without making any assumption on the true data-generating process. The sensitivity still depends on the partial R2 of unobserved confounder wrt. treatment and outcome, $\eta^2_{T\sim U | W}$ and $\eta^2_{Y \sim U | T, W}$ respectively, but the computation of bounds is more complicated and requires estimation of a special function known as reisz function. Refer to Chernozhukov et al. for details.

In [ ]:
# Estimate effect using a non-parametric estimator
from sklearn.ensemble import GradientBoostingRegressor
estimate_npar = model.estimate_effect(identified_estimand, 
                                    method_name="backdoor.econml.dml.KernelDML",
                                    method_params={
                                        'init_params': {'model_y':GradientBoostingRegressor(),
                                                        'model_t': GradientBoostingRegressor(),                                                       },
                                        'fit_params': {},
                                     })
print(estimate_npar)

To do the sensitivity analysis, we now use the same `non-parametric--partial-R2` method, however the estimation of partial R2 will be based on reisz representers. We use `plugin_reisz=True` to specify that we will be using a plugin reisz function estimator (this is faster and available for binary treatments). Otherwise, we can set it to False to estimate reisz function separately using a loss function.

In [ ]:
refute_npar = model.refute_estimate(identified_estimand, estimate_npar,
                               method_name = "add_unobserved_common_cause",
                               simulation_method = "non-parametric-partial-R2",
                               benchmark_common_causes = ["W1"],
                               effect_fraction_on_treatment = 0.2,
                               effect_fraction_on_outcome = 0.2,
                               plugin_reisz=True
                              )
print(refute_npar)

The plot has the same interpretation as before. 


To verify, the true causal effect can be seen below. 

In [ ]:
refute.S2

##### Parameter List for plot function
- <b>plot_type</b>: possible values are 'bias','lower_ate_bound','upper_ate_bound','lower_confidence_bound','upper_confidence_bound'<br>
- <b>x_limit</b>: plot's maximum x_axis value (default = 0.8) <br>
- <b>y_limit</b>: plot's minimum y_axis value (default = 0.8) <br>
- <b>num_points_per_contour</b>: number of points to calculate and plot each contour line (default = 200) <br>
- <b>plot_size</b>: tuple denoting the size of the plot (default = (7,7))<br>
- <b>contours_color</b>: color of contour line (default = blue)<br>
String or array. If array, lines will be plotted with the specific color in ascending order.<br>
- <b>critical_contour_color</b>: color of threshold line (default = red)<br>
- <b>label_fontsize</b>: fontsize for labelling contours (default = 9)<br>
- <b>contour_linewidths</b>: linewidths for contours (default = 0.75)<br>
- <b>contour_linestyles</b>: linestyles for contours (default = "solid") See : https://matplotlib.org/3.5.0/gallery/lines_bars_and_markers/linestyles.html<br>
- <b>contours_label_color</b>: color of contour line label (default = black)<br>
- <b>critical_label_color</b>: color of threshold line label (default = red)<br>
- <b>unadjusted_estimate_marker</b>: marker type for unadjusted estimate in the plot (default = 'D')
See: https://matplotlib.org/stable/api/markers_api.html <br>
- <b>unadjusted_estimate_color</b>: marker color for unadjusted estimate in the plot (default = "black")<br>
- <b>adjusted_estimate_marker</b>: marker type for bias adjusted estimates in the plot (default = '^')<br>
- <b>adjusted_estimate_color</b>: marker color for bias adjusted estimates in the plot (default = "red")<br>
- <b>legend_position</b>:tuple denoting the position of the legend (default = (1.6, 0.6))<br>

In [ ]:
refute.plot(plot_type = "upper_ate_bound")

In [ ]:
refute.plot(plot_type = "bias")

In [ ]:
refute.results

In [ ]:
print(refute)

The robustness value measures the minimal equal strength of r2yu_tw and r2tu_w such the bound for the average treatment effect would include zero. It can be between 0 and 1.<br>
A low robustness value implies that the results can be changed even by the presence of weak confounders whereas a robustness value close to 1 means the treatment effect can handle strong confounders explaining  almost all residual variation of the treatment and the outcome.
A robustness value of 0.68 implies that confounders with r2yu_tw and r2tu_w values less than 0.68 would not be sufficient enough to bring down the estimates to zero.